In [44]:
import mysql.connector as con
import pandas as pd
import random
import datetime 
from datetime import datetime, timedelta


## ENUNCIADO EJERCICIO 

### 1. CREAR BASE DE DATOS (33%)

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.





In [45]:
def crear_tabla(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [46]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [47]:
crear_tabla(sql)

## Crear TABLAS

![alt text](image.png)

tiendas
- id_tienda (PRIMARY KEY)
- nombre_tienda
- direccion
- ciudad

In [48]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(255) NOT NULL, direccion VARCHAR(255), ciudad VARCHAR(255) NOT NULL);"

In [49]:
crear_tabla(sql)

empleados
- id_empleado (PRIMARY KEY)
- nombre_empleado
- puesto (ej.: Cajero, Gerente, Reponedor)
- id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [50]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(255) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [51]:
crear_tabla(sql)

categorias
- id_categoria (PRIMARY KEY)
- nombre_categoria

In [52]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [53]:
crear_tabla(sql)

productos
- id_producto (PRIMARY KEY)
- nombre_producto
- precio
- stock
- id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [54]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [55]:
crear_tabla(sql)

clientes
- id_cliente (PRIMARY KEY)
- first_name
- last_name
- email
- codigo_postal

In [56]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(200), codigo_postal INT);"

In [57]:
crear_tabla(sql)

ordenes
- id_orden (PRIMARY KEY)
- id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
- id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
- fecha_orden
- metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [58]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [59]:
crear_tabla(sql)

detalle_orden
- id_detalle (PRIMARY KEY)
- id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
- id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
- cantidad
- precio_unitario: mismo precio que en la tabla producto
- descuento (podría ser NULL si no se aplica)

In [60]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [61]:
crear_tabla(sql)

### 2. Generar datos demo desde Python (33%)

* Generar datos aleatorios en listas con Python similar los realizados en clase.
    * Uso de datetime, timedelta, random para generar datos aleatorios
* Pasar los datos a DataFrames de Pandas
* Pasar los DataFrames de Pandas a MySQL usando la función to_sql de Pandas con SQLAlchemy o usando MySQL Connector con sentencias INSERT.

In [62]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

tiendas

* id_tienda: valores enteros consecutivos (por ejemplo, 1, 2, 3...).
* nombre_tienda: nombres ficticios o genéricos (p. ej. "Tienda Centro", "Super Norte", "Super Sur").
* direccion: direcciones simples (p. ej. “Calle Falsa 123”).
* ciudad: usar nombres de ciudades ficticias o reales (p. ej. “Madrid”, “Barcelona”, “México DF”, etc.).

En total 5 o 10 tiendas.

In [63]:
tiendas = ['Jerez Este', 'Sanlucar Oeste', 'Cadiz Norte', 'Valdelagrana Sur', 'Arcos Sur', 'Chiclana Norte', 'Barbate Este', 'Vejer Oeste', 'Conil Sur', 'Rota Norte']
direcciones = ['Calle Catavino 25', 'Calle Manzanilla 42', 'Calle del Pan 38', 'Calle Playa 57', 'Calle Castillo 182', 'Calle Barrosa 63', 'Calle Atun 98', 'Calle Pescadill 56', 'Calle Balsa 71', 'Calle Base 22']
ciudades = ['Jerez de la Frontera', 'Sanlucar de Barrameda', 'Cadiz', 'El Puerto de Santa Maria', 'Arcos de la Frontera', 'Chiclana de la Frontera', 'Barbate', 'Vejer de la Frontera', 'Conil de la Frontera', 'Rota']
df_tiendas = pd.DataFrame(zip(tiendas, direcciones, ciudades), columns=['nombre_tienda', 'direccion', 'ciudad'])
# generar columna id_tienda
df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas

,id_tienda,nombre_tienda,direccion,ciudad
0,1,Jerez Este,Calle Catavino 25,Jerez de la Frontera
1,2,Sanlucar Oeste,Calle Manzanilla 42,Sanlucar de Barrameda
2,3,Cadiz Norte,Calle del Pan 38,Cadiz
3,4,Valdelagrana Sur,Calle Playa 57,El Puerto de Santa Maria
4,5,Arcos Sur,Calle Castillo 182,Arcos de la Frontera
5,6,Chiclana Norte,Calle Barrosa 63,Chiclana de la Frontera
6,7,Barbate Este,Calle Atun 98,Barbate
7,8,Vejer Oeste,Calle Pescadill 56,Vejer de la Frontera
8,9,Conil Sur,Calle Balsa 71,Conil de la Frontera
9,10,Rota Norte,Calle Base 22,Rota


In [64]:
insertar_dataframe(df_tiendas, 'tiendas')

10

empleados

* id_empleado: valores enteros consecutivos (1, 2, 3...).
* nombre_empleado: nombres y apellidos ficticios (p. ej. “Laura Gutiérrez”, “Juan Pérez”).
* puesto: limitarse a un conjunto reducido de valores (p. ej. {‘Cajero’, ‘Gerente’, ‘Reponedor’, ‘Vendedor’}).
* id_tienda: debe hacer referencia a una tienda existente (por ejemplo, un número entre 1 y 3 si solo tienes 3 tiendas).

En total 20 empleados por tienda.

In [65]:
#generar columna id_empleado
empleados = list()
nombres = ["Juan", "María", "Luis", "Ana", "Pedro", "Sofía", "Carlos", "Lucía", "Jorge", "Elena", "Manuel", "Laura", "Diego", "Clara", "Alberto", "Teresa", "Pablo", "Isabel", "Sergio", "Marta", "Roberto", "Julia", "Fernando", "Carmen", "Ricardo", "Paula", "Andrés", "Eva", "Antonio", "Sara", "José", "Natalia", "Francisco", "Patricia", "Hugo", "Adriana", "Rafael", "Irene", "Víctor", "Olga", "Daniel", "Noelia", "Álvaro", "Alicia", "Javier", "Rocío", "Enrique", "Amelia", "Gonzalo", "Vera"]
apellidos = ["García", "Martínez", "López", "Sánchez", "Pérez", "González", "Rodríguez", "Fernández", "Moreno", "Jiménez", "Hernández", "Muñoz", "Álvarez", "Romero", "Alonso", "Gutiérrez", "Ruiz", "Navarro", "Díaz", "Torres", "Castro", "Vázquez", "Ramos", "Gil", "Ramírez", "Flores", "Ortiz", "Marín", "Serrano", "Iglesias", "Delgado", "Domínguez", "Guerrero", "Molina", "Ortega", "Rubio", "Santos", "Medina", "Castillo", "Cruz", "Reyes", "Herrera", "Peña", "Carrasco", "Calvo", "Vega", "Cabrera", "Campos", "León", "Fuentes"]
trabajo = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Responsable de almacén", "Supervisor de caja", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de inventario", "Responsable de compras", "Promotor de ventas", "Encargado de seguridad"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = trabajo[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados

,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Juan Martínez,Cajero,1
1,2,Hugo Carrasco,Reponedor,1
2,3,Hugo Medina,Encargado de sección,1
3,4,Paula Alonso,Gerente,1
4,5,Fernando Medina,Personal de limpieza,1
...,...,...,...,...
195,196,Fernando Gutiérrez,Auxiliar de caja,10
196,197,Noelia García,Encargado de inventario,10
197,198,Alberto Santos,Responsable de compras,10
198,199,Carlos Vega,Promotor de ventas,10


In [66]:
insertar_dataframe(df_empleados, "empleados")

200

categorias

* id_categoria: valores enteros consecutivos (1, 2, 3...).
* nombre_categoria: selección de categorías (p. ej. “Lácteos”, “Carnes”, “Frutas”, “Verduras”, “Bebidas”, “Snacks”).

En total 10 categorías

In [67]:
categorias = ['Lácteos', 'Carnes', 'Frutas', 'Verduras', 'Bebidas', 'Snacks', 'Hogar', 'Limpieza', 'Perfumería', 'Jardín']

df_categorias = pd.DataFrame(categorias, columns=['nombre_categoria'])

# generar columna id_categoria
df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias

,id_categoria,nombre_categoria
0,1,Lácteos
1,2,Carnes
2,3,Frutas
3,4,Verduras
4,5,Bebidas
5,6,Snacks
6,7,Hogar
7,8,Limpieza
8,9,Perfumería
9,10,Jardín


In [68]:
insertar_dataframe(df_categorias, 'categorias')

10


productos

* id_producto: valores enteros consecutivos (1, 2, 3...).
* nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
* precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
* stock: valores INT entre 0 y 500 (o el rango que quieras).
* id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).

En total 4 productos de cada categoría.

In [69]:
# generar columna id_producto
produ = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(produ)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 100))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(produ, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,1.776551,4,1
1,2,Yogur,10.403783,22,1
2,3,Queso,8.399610,75,1
3,4,Mantequilla,2.478085,48,1
4,5,Pollo,3.732859,84,2
5,6,Carne de res,2.351971,8,2
6,7,Cerdo,6.950004,85,2
7,8,Chuletas,6.585489,23,2
8,9,Manzana,1.553522,63,3
9,10,Banana,5.543199,9,3


In [70]:
insertar_dataframe(df_productos, 'productos')

40

clientes

* id_cliente: valores enteros consecutivos (1, 2, 3...).
* nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
* email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
* telefono: número de 9 o 10 dígitos (dependiendo del país).
* direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).

En total: 2000.


In [71]:
# generar columna id_cliente
nombre_cliente, apellido_cliente, email, postal, cliente_completo = list(), list(), list(), list(), list()
calles = ("Av. Libertad", "Calle Mayor", "Paseo de la Reforma", "Calle Real", "Av. de la Constitución", "Calle del Sol", "Calle de la Luna", "Calle los Pinos", "Calle de la Paz", "Calle Olivo", "Calle Cedro", "Calle Nogal", "Av. de las Flores", "Paseo del Prado", "Calle del Río", "Av. Central", "Calle Primavera", "Calle Otoño", "Calle Invierno", "Calle Verano", "Calle Granada", "Calle Sevilla", "Calle Córdoba", "Calle Málaga", "Calle Valencia", "Calle Zaragoza", "Calle Alicante", "Av. de los Ángeles", "Calle del Carmen", "Calle San Miguel", "Calle San Juan", "Calle del Pilar", "Calle Santa Teresa", "Calle San Francisco", "Calle de la Sierra", "Calle del Mar", "Av. del Norte", "Av. del Sur", "Calle del Este", "Calle del Oeste", "Calle Diamante", "Calle Esmeralda", "Calle Rubí", "Calle Zafiro", "Calle Ámbar", "Calle Turquesa", "Calle Topacio", "Calle Perla", "Calle Horizonte", "Calle Amanecer")
for _ in range(2000):
    nom = random.choice(nombres)
    nombre_cliente.append(nom)
    ape = random.choice(apellidos)
    apellido_cliente.append(ape)
    email.append((f'{nom}.{ape}@suemail.com'))
    postal.append(random.randint(11000, 11900))
    
df_clientes = pd.DataFrame(zip(nombre_cliente, apellido_cliente, email, postal), columns=['first_name', 'last_name', 'email', 'codigo_postal']) 
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,first_name,last_name,email,codigo_postal
0,1,Isabel,Guerrero,Isabel.Guerrero@suemail.com,11774
1,2,Sara,Castro,Sara.Castro@suemail.com,11132
2,3,Ricardo,Campos,Ricardo.Campos@suemail.com,11803
3,4,Antonio,Campos,Antonio.Campos@suemail.com,11260
4,5,Alicia,Vázquez,Alicia.Vázquez@suemail.com,11055
...,...,...,...,...,...
1995,1996,Carmen,González,Carmen.González@suemail.com,11205
1996,1997,Roberto,Flores,Roberto.Flores@suemail.com,11571
1997,1998,Clara,Gutiérrez,Clara.Gutiérrez@suemail.com,11237
1998,1999,Sofía,Ruiz,Sofía.Ruiz@suemail.com,11503


In [72]:
insertar_dataframe(df_clientes, 'clientes')

2000

ordenes

* id_orden: valores enteros consecutivos (1, 2, 3...).
* id_cliente: haz referencia a los IDs existentes de la tabla clientes.
* id_empleado: haz referencia a los IDs existentes de la tabla empleados.
* fecha_orden: genera fechas aleatorias o secuenciales (p. ej. entre ‘2024-01-01’ y ‘2025-01-01’).
* metodo_pago: escoge entre {‘Tarjeta’, ‘Efectivo’}.

En total: 10000.


In [73]:
# generar columna id_orden
cliente, emplea, fecha, metodo = list(), list(), list(), list()
fechas = [datetime(year=2024, month=1, day=1) + timedelta(days=numero) for numero in range(395)]
for _ in range(10000):
    cliente.append(random.choice(df_clientes['id_cliente']))
    emplea.append(random.choice(df_empleados['id_empleado']))
    fecha.append(random.choice(fechas))
    metodo.append(random.choices(['Tarjeta', 'Efectivo'], weights=[0.65, 0.35])[0])

fecha.sort()
df_ordenes = pd.DataFrame(zip(cliente, emplea, fecha, metodo), columns=['id_cliente', 'id_empleado', 'fecha_orden', 'metodo_pago'])
df_ordenes.sort_values('fecha_orden', inplace=True)
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes.sort_values('id_orden', inplace=True)
df_ordenes.head(50)

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,1787,27,2024-01-01,Tarjeta
22,2,543,165,2024-01-01,Tarjeta
17,3,725,58,2024-01-01,Tarjeta
18,4,816,132,2024-01-01,Efectivo
19,5,371,53,2024-01-01,Efectivo
20,6,1412,129,2024-01-01,Tarjeta
21,7,933,92,2024-01-01,Tarjeta
28,8,1088,50,2024-01-01,Tarjeta
23,9,306,57,2024-01-01,Efectivo
24,10,775,123,2024-01-01,Tarjeta


In [74]:
insertar_dataframe(df_ordenes, 'ordenes')

10000

detalle_orden

* id_detalle: valores enteros consecutivos (1, 2, 3...).
* id_orden: referencia al ID de alguna orden válida.
* id_producto: referencia al ID de algún producto válido.
* cantidad: valores entre 1 y 20, por ejemplo.
* precio_unitario: usar el mismo precio que está en la tabla productos o uno ligeramente distinto si quieres simular ofertas.
* descuento: valores DECIMAL bajos (p. ej. 0.00, 1.00, 2.50) o NULL.

En total: 30000.

In [75]:
orden_completa = list()
productos = df_productos['id_producto']
orden_20 = df_ordenes['id_orden']
for orden in df_ordenes['id_orden']:
    ordenes = orden
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor, 2), round(descuento, 2)])   
for orden in range(20000):
    ordenes = random.choice(orden_20)
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor,2), round(descuento, 2)])   
df_detalle_orden = pd.DataFrame(orden_completa, columns=['id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento'])
df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,descuento
0,1,1,4,11,2.48,0.05
1,2,2,16,14,7.03,0.13
2,3,3,16,16,7.03,0.18
3,4,4,13,3,1.89,0.01
4,5,5,2,6,10.40,0.10
...,...,...,...,...,...,...
29995,29996,1151,31,8,0.76,0.10
29996,29997,222,28,13,6.01,0.12
29997,29998,5443,19,18,3.08,0.10
29998,29999,6191,12,7,7.39,0.02


In [76]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

30000

### 3. Consultas SQL (34%)

In [77]:
def consultas(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="admin",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
        return cursor.fetchall()
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

1. Listado de órdenes con detalles de cliente y empleado
* Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
* Utiliza un JOIN entre las tablas ordenes, clientes y empleados.

In [78]:
sql = ('SELECT o.id_orden, o.fecha_orden, c.first_name, c.last_name, e.nombre_empleado, o.metodo_pago FROM ordenes o JOIN clientes c ON c.id_cliente = o.id_cliente JOIN empleados e ON e.id_empleado = o.id_empleado ORDER BY o.id_orden;') 
consultas(sql)

[(1, datetime.date(2024, 1, 1), 'Alicia', 'Ortiz', 'Ana Molina', 'Tarjeta'),
 (2, datetime.date(2024, 1, 1), 'Elena', 'Cruz', 'Enrique Campos', 'Tarjeta'),
 (3,
  datetime.date(2024, 1, 1),
  'Rocío',
  'Sánchez',
  'María Serrano',
  'Tarjeta'),
 (4,
  datetime.date(2024, 1, 1),
  'Julia',
  'González',
  'Teresa Carrasco',
  'Efectivo'),
 (5,
  datetime.date(2024, 1, 1),
  'Enrique',
  'Fuentes',
  'José Gutiérrez',
  'Efectivo'),
 (6, datetime.date(2024, 1, 1), 'Clara', 'Rubio', 'Pedro León', 'Tarjeta'),
 (7, datetime.date(2024, 1, 1), 'Sergio', 'Fuentes', 'Ana Medina', 'Tarjeta'),
 (8,
  datetime.date(2024, 1, 1),
  'Alicia',
  'Carrasco',
  'Roberto Gutiérrez',
  'Tarjeta'),
 (9,
  datetime.date(2024, 1, 1),
  'Carmen',
  'Gil',
  'Marta Fernández',
  'Efectivo'),
 (10,
  datetime.date(2024, 1, 1),
  'Pablo',
  'Reyes',
  'Isabel Rodríguez',
  'Tarjeta'),
 (11,
  datetime.date(2024, 1, 1),
  'Daniel',
  'Hernández',
  'Olga Castro',
  'Efectivo'),
 (12,
  datetime.date(2024, 1, 1)

2. Productos con stock bajo
* Filtra aquellos productos cuyo stock sea menor a 10.
* Muestra nombre del producto, categoría y stock.


In [79]:
sql = ('SELECT p.nombre_producto, c.nombre_categoria, p.stock FROM productos p JOIN categorias c ON p.id_categoria = c.id_categoria WHERE p.stock < 10;')
consultas(sql)

[('Leche', 'Lácteos', 4),
 ('Carne de res', 'Carnes', 8),
 ('Banana', 'Frutas', 9),
 ('Cebolla', 'Verduras', 3),
 ('Crema hidratante', 'Perfumería', 0)]

3. Ventas totales por categoría
* Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
* Realiza el JOIN con detalle_orden, productos y categorias.
* Utiliza agrupación (GROUP BY).

In [80]:
sql = ('SELECT c.nombre_categoria as categoria, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as ventas_netas, sum(d.precio_unitario * d.cantidad) as ventas_brutas FROM categorias c JOIN productos p ON p.id_categoria = c.id_categoria JOIN detalle_orden d ON d.id_producto = p.id_producto group by c.id_categoria;')
consultas(sql)

[('Lácteos', 182978.77, 182978.77),
 ('Verduras', 197951.31, 197951.31),
 ('Perfumería', 224949.63, 224949.63),
 ('Snacks', 250437.05, 250437.05),
 ('Bebidas', 167004.61, 167004.61),
 ('Hogar', 201490.72, 201490.72),
 ('Limpieza', 115963.72, 115963.72),
 ('Frutas', 157637.68, 157637.68),
 ('Carnes', 154848.99, 154848.99),
 ('Jardín', 233541.8, 233541.8)]

4. Clientes con mayores gastos acumulados
* Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
* Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
* Ordena el resultado de mayor a menor gasto acumulado.

In [81]:
sql = ('SELECT c.first_name as nombre, c.last_name as apellido, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as compras FROM clientes c JOIN ordenes o ON c.id_cliente = o.id_cliente JOIN detalle_orden d ON d.id_orden = o.id_orden GROUP BY c.id_cliente ORDER BY sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) DESC;')
consultas(sql)

[('Rafael', 'Castro', 3179.96),
 ('Luis', 'Reyes', 2893.73),
 ('Antonio', 'Molina', 2720.4),
 ('Pedro', 'Torres', 2592.0),
 ('Adriana', 'Navarro', 2452.5),
 ('Andrés', 'Gil', 2420.94),
 ('Julia', 'Álvarez', 2389.0),
 ('Patricia', 'Guerrero', 2360.06),
 ('Rafael', 'Campos', 2335.47),
 ('Amelia', 'Serrano', 2326.06),
 ('Gonzalo', 'Gil', 2324.47),
 ('Juan', 'Cabrera', 2321.55),
 ('Antonio', 'Ramos', 2317.14),
 ('Rocío', 'Gutiérrez', 2313.11),
 ('Rafael', 'Peña', 2306.91),
 ('Hugo', 'Reyes', 2304.37),
 ('Antonio', 'Alonso', 2299.13),
 ('Javier', 'Medina', 2297.55),
 ('Lucía', 'Hernández', 2287.58),
 ('Hugo', 'Pérez', 2278.05),
 ('Antonio', 'Molina', 2256.25),
 ('Amelia', 'Fuentes', 2252.77),
 ('Pedro', 'Marín', 2246.08),
 ('Alicia', 'González', 2230.89),
 ('Andrés', 'Iglesias', 2224.12),
 ('Alicia', 'Díaz', 2223.91),
 ('Irene', 'Reyes', 2190.32),
 ('Clara', 'Molina', 2183.13),
 ('Julia', 'Cabrera', 2180.29),
 ('Javier', 'Castillo', 2166.08),
 ('Laura', 'Rodríguez', 2140.28),
 ('Lucía', 'Ca

5. Empleados y número de órdenes gestionadas
* Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
* Utiliza GROUP BY y COUNT.

In [82]:
sql = ('SELECT e.nombre_empleado AS EMPLEADO, e.puesto AS PUESTO, count(o.id_empleado) AS ORDENES FROM empleados e JOIN ordenes o ON o.id_empleado = e.id_empleado GROUP BY e.id_empleado')
consultas(sql)

[('Juan Martínez', 'Cajero', 58),
 ('Hugo Carrasco', 'Reponedor', 69),
 ('Hugo Medina', 'Encargado de sección', 43),
 ('Paula Alonso', 'Gerente', 44),
 ('Fernando Medina', 'Personal de limpieza', 43),
 ('Luis Muñoz', 'Panadero', 63),
 ('Gonzalo Carrasco', 'Carnicero', 52),
 ('Carlos Martínez', 'Pescadero', 49),
 ('Vera Hernández', 'Responsable de almacén', 50),
 ('Clara Hernández', 'Supervisor de caja', 50),
 ('Marta Cruz', 'Frutero', 50),
 ('Paula Gil', 'Charcutero', 41),
 ('Teresa Rubio', 'Jefe de turno', 51),
 ('José Marín', 'Operador logístico', 52),
 ('Álvaro Rubio', 'Atención al cliente', 51),
 ('Roberto Herrera', 'Auxiliar de caja', 58),
 ('Rafael Herrera', 'Encargado de inventario', 46),
 ('Alberto Díaz', 'Responsable de compras', 42),
 ('Francisco Carrasco', 'Promotor de ventas', 54),
 ('Fernando Ortega', 'Encargado de seguridad', 44),
 ('Alicia Serrano', 'Cajero', 49),
 ('Álvaro García', 'Reponedor', 52),
 ('Francisco González', 'Encargado de sección', 55),
 ('Sergio López', 

6. Ordenes filtradas por fecha y tienda
* Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
* Incluye datos de la tienda y del cliente.

In [83]:
sql = ("SELECT t.id_tienda, t.nombre_tienda AS tienda, o.id_orden AS factura, o.fecha_orden AS fecha, concat(c.first_name,' ',c.last_name) AS cliente FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON o.id_empleado = e.id_empleado JOIN clientes c ON c.id_cliente = o.id_cliente WHERE T.id_tienda = 1 and o.fecha_orden > '2024-01-31' and o.fecha_orden < '2024-03-01' ORDER BY o.fecha_orden;")
consultas(sql)

[(1, 'Jerez Este', 804, datetime.date(2024, 2, 1), 'Noelia Marín'),
 (1, 'Jerez Este', 810, datetime.date(2024, 2, 1), 'Luis Ramos'),
 (1, 'Jerez Este', 821, datetime.date(2024, 2, 1), 'Clara Fernández'),
 (1, 'Jerez Este', 826, datetime.date(2024, 2, 2), 'María Reyes'),
 (1, 'Jerez Este', 827, datetime.date(2024, 2, 2), 'Isabel Vega'),
 (1, 'Jerez Este', 832, datetime.date(2024, 2, 2), 'Paula Moreno'),
 (1, 'Jerez Este', 855, datetime.date(2024, 2, 2), 'Javier Martínez'),
 (1, 'Jerez Este', 861, datetime.date(2024, 2, 3), 'Laura Flores'),
 (1, 'Jerez Este', 875, datetime.date(2024, 2, 3), 'María Castillo'),
 (1, 'Jerez Este', 886, datetime.date(2024, 2, 3), 'Olga Pérez'),
 (1, 'Jerez Este', 888, datetime.date(2024, 2, 3), 'Víctor Ramos'),
 (1, 'Jerez Este', 905, datetime.date(2024, 2, 4), 'Elena Navarro'),
 (1, 'Jerez Este', 918, datetime.date(2024, 2, 5), 'Sara Castro'),
 (1, 'Jerez Este', 925, datetime.date(2024, 2, 5), 'Rafael Ramírez'),
 (1, 'Jerez Este', 936, datetime.date(2024, 

7. Ranking de productos más vendidos en cada tienda
* Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
* Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
* Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).


In [84]:
#CON MUCHA AYUDA!!!!
sql = ('WITH ranking_productos AS (SELECT t.nombre_tienda, p.nombre_producto, SUM(do.cantidad) AS total_vendido, ROW_NUMBER() OVER (PARTITION BY t.id_tienda ORDER BY SUM(do.cantidad) DESC) AS ranking FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON e.id_empleado = o.id_empleado JOIN detalle_orden do ON o.id_orden = do.id_orden JOIN productos p ON do.id_producto = p.id_producto GROUP BY t.id_tienda, t.nombre_tienda, p.id_producto, p.nombre_producto) SELECT nombre_tienda, nombre_producto, total_vendido FROM ranking_productos WHERE ranking <= 3 ORDER BY nombre_tienda, ranking;')
consultas(sql)

[('Arcos Sur', 'Mantequilla', 998.0),
 ('Arcos Sur', 'Pollo', 985.0),
 ('Arcos Sur', 'Zanahoria', 974.0),
 ('Barbate Este', 'Leche', 1029.0),
 ('Barbate Este', 'Cerveza', 1028.0),
 ('Barbate Este', 'Cebolla', 982.0),
 ('Cadiz Norte', 'Tomate', 978.0),
 ('Cadiz Norte', 'Colonia', 909.0),
 ('Cadiz Norte', 'Chuletas', 865.0),
 ('Chiclana Norte', 'Leche', 1050.0),
 ('Chiclana Norte', 'Bolsas de basura', 1044.0),
 ('Chiclana Norte', 'Platos desechables', 1033.0),
 ('Conil Sur', 'Chocolate', 924.0),
 ('Conil Sur', 'Yogur', 893.0),
 ('Conil Sur', 'Cebolla', 874.0),
 ('Jerez Este', 'Fertilizante', 979.0),
 ('Jerez Este', 'Papas fritas', 973.0),
 ('Jerez Este', 'Zanahoria', 949.0),
 ('Rota Norte', 'Lechuga', 990.0),
 ('Rota Norte', 'Desodorante', 971.0),
 ('Rota Norte', 'Semillas', 949.0),
 ('Sanlucar Oeste', 'Pilas', 991.0),
 ('Sanlucar Oeste', 'Crema hidratante', 949.0),
 ('Sanlucar Oeste', 'Banana', 930.0),
 ('Valdelagrana Sur', 'Esponjas', 930.0),
 ('Valdelagrana Sur', 'Uva', 928.0),
 ('Val

Opcional: añadir alguna consulta con subconsultas o algo que no se abarque en las anteriores consulta.
*  Mostrar los clientes que han comprado más de la media de productos vendidos

In [85]:
sql = ('SELECT c.first_name AS nombre_cliente, c.last_name AS apellido_cliente, COUNT(do.id_producto) AS productos_comprados FROM clientes c JOIN ordenes o ON c.id_cliente = o.id_cliente JOIN detalle_orden do ON o.id_orden = do.id_orden GROUP BY c.id_cliente, c.first_name, c.last_name HAVING productos_comprados > (SELECT AVG(cantidad_total) FROM (SELECT SUM(do.cantidad) AS cantidad_total FROM detalle_orden do GROUP BY do.id_orden) AS subquery) ORDER BY productos_comprados DESC;')
consultas(sql)

[('Luis', 'Reyes', 49),
 ('Rafael', 'Castro', 42),
 ('Hugo', 'Reyes', 39),
 ('Elena', 'Delgado', 39),
 ('Rafael', 'Campos', 38),
 ('Sara', 'Gutiérrez', 37),
 ('Paula', 'Gutiérrez', 37),
 ('Antonio', 'Molina', 37),
 ('Lucía', 'Hernández', 36),
 ('Alicia', 'Díaz', 36),
 ('Antonio', 'Alonso', 36),
 ('Andrés', 'Gil', 36),
 ('Andrés', 'Castillo', 35),
 ('Juan', 'Cabrera', 35),
 ('Pedro', 'Torres', 35),
 ('Antonio', 'Vázquez', 35),
 ('Ana', 'Guerrero', 35),
 ('Pablo', 'Martínez', 35),
 ('Isabel', 'Castillo', 34),
 ('Clara', 'Molina', 34),
 ('Alicia', 'González', 34),
 ('Teresa', 'Flores', 34),
 ('Carmen', 'Serrano', 34),
 ('Irene', 'Reyes', 34),
 ('Roberto', 'Ruiz', 33),
 ('Gonzalo', 'Gil', 33),
 ('Rocío', 'Gutiérrez', 33),
 ('Enrique', 'Álvarez', 33),
 ('Amelia', 'Serrano', 33),
 ('Amelia', 'Martínez', 33),
 ('Julia', 'Alonso', 33),
 ('Andrés', 'Iglesias', 33),
 ('Julia', 'Álvarez', 33),
 ('Laura', 'Rodríguez', 32),
 ('Sergio', 'Álvarez', 32),
 ('Noelia', 'López', 32),
 ('Lucía', 'Castillo'